In [1]:
import pandas as pd 
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import calendar
from datetime import date
import holidays
from pandas.tseries.offsets import CustomBusinessDay
import numpy as np

    
# Define o escopo
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']

# Autenticação com o arquivo de credenciais
creds = ServiceAccountCredentials.from_json_keyfile_name('C:/Users/novo1/OneDrive/Desktop/Dev/Dashboard - Mentoria/dashboard-mentoria-465319-21cfa3848fa4.json', scope)
client = gspread.authorize(creds)

# Abrindo as Planilhas com os dados
planilha_respostas = client.open_by_url("https://docs.google.com/spreadsheets/d/1V1d0MsCQxT-a4yChAtLfzVuiXyTzjjiFqw37yASqlmE/edit?gid=1498245696#gid=1498245696")
planilha_controle = client.open_by_url("https://docs.google.com/spreadsheets/d/1BvFVsdi5NBkoA7Lyq64b6T7_ZGBZYBQ5ONTTfWuzB0s/edit?resourcekey=&gid=706238849#gid=706238849")

# Pegando a aba com as informações de Mentorados
aba_control = planilha_controle.worksheet("Forms Control")
dados_control = aba_control.get_all_records()

# Transformando em dataframe
df_controle = pd.DataFrame(dados_control)

# Criando a lista de mentorados
df_controle = df_controle.loc[df_controle["CLÍNICA"] != 'Padrão']
lista_mentorados = df_controle["CLÍNICA"].unique().tolist()

lista_de_dfs = []

for clinic in lista_mentorados:
    aba_respostas = planilha_respostas.worksheet(clinic)
    dados_resposta = aba_respostas.get_all_records()

    df_respostas = pd.DataFrame(dados_resposta)

    df_respostas["Clinica"] = clinic

    lista_de_dfs.append(df_respostas)
    print(f"adicionando : {clinic}")
print(f"Todas clinicas adicionadas")
df_final = pd.concat(lista_de_dfs, ignore_index=True)

#Treating Dataframe: 

df_final_columns = ['Clinica','Data','Qual a sua Meta de Faturamento?', 'Leads Gerados no Dia:',
                    'Avaliações Realizadas no Dia:','Atendimentos Realizados no dia.\n(considerando Avaliação)',
                    'Quantidade de Pedidos Gerados no DIa:','Valor Vendido no Dia (somente número):']

df_final = df_final[df_final_columns]

df_final


adicionando : MITRAS ESTÉTICA
adicionando : T DANTAS DOS SANTOS LTDA
adicionando : CLÍNICA KAYA
adicionando : BARRUFINI E VIDIGAL LTDA - ESPAÇO LASER
adicionando : CLÍNICA DRA JESSICA CRUZ
adicionando : ATUAL ESTÉTICA E IMAGEM PESSOAL
adicionando : CLAUDIA PELUCO ESTÉTICA E SAÚDE LTDA
adicionando : CENTRO DE ESTÉTICA DANIELA DA ROSA
adicionando : MSP ESTETICA LTDA
adicionando : DENT.AI
adicionando : RAFAELA FERNANDES SERVIOS MÉDICOS
adicionando : INSTITUTO PELE E SAUDE
adicionando : BEAUTY GLOW ACADEMY
adicionando : INSTITUTO DERME
adicionando : JESSILY RIBAS TORRES DE AZEVEDO LTDA
adicionando : INSTITUTO AZURI DE ESTETICA
adicionando : ALMEIDA E GIRON
adicionando : TTDS ODONTOLOGIA E EDUCACAO LTDA
adicionando : LA FEMME LTDA
adicionando : BLUE CLINIC LTDA
adicionando : EVUP TECNOLOGIA DA INFORMACAO LTDA
adicionando : HARMONISER
Todas clinicas adicionadas


,Clinica,Data,Qual a sua Meta de Faturamento?,Leads Gerados no Dia:,Avaliações Realizadas no Dia:,Atendimentos Realizados no dia.\n(considerando Avaliação),Quantidade de Pedidos Gerados no DIa:,Valor Vendido no Dia (somente número):
0,MITRAS ESTÉTICA,7/1/2025,50000,5,5,8,10,1871
1,MITRAS ESTÉTICA,7/2/2025,50000,10,7,15,10,1139
2,MITRAS ESTÉTICA,7/3/2025,50000,15,5,6,10,1654
3,MITRAS ESTÉTICA,7/4/2025,50000,20,11,14,5,1484
4,MITRAS ESTÉTICA,7/5/2025,50000,25,6,12,10,1292
...,...,...,...,...,...,...,...,...
655,HARMONISER,7/26/2025,50000,50,7,6,9,687
656,HARMONISER,7/27/2025,50000,55,9,10,6,661
657,HARMONISER,7/28/2025,50000,5,6,14,10,849
658,HARMONISER,7/29/2025,50000,20,5,7,10,985


In [ ]:
# Data atual
today = date.today()
year = today.year
month = today.month

# start date and end date
start_date = f'{year}-{month:02d}-01'
end_day = pd.Timestamp(year=year, month=month, day=1).days_in_month
end_date = f'{year}-{month:02d}-{end_day}'

# Brazilian holidays
br_holidays = holidays.CountryHoliday('BR', years=[year])

# converting the holidays to list
holiday_dates = list(br_holidays.keys())

# creating a custom businnes days
custom_bday = CustomBusinessDay(holidays=holiday_dates, weekmask='Mon Tue Wed Thu Fri Sat')

# generate business days
business_days = pd.date_range(start=start_date, end=end_date, freq=custom_bday)
num_business_day = len(business_days)

# groupby to get the sales by day by mentee
venda_diaria_gp = df_final.groupby(['Clinica','Data']).agg({"Valor Vendido no Dia (somente número):" : 'sum'}).reset_index()

# treating date values
venda_diaria_gp['Data'] = pd.to_datetime(venda_diaria_gp['Data'])

# sorting values by date
venda_diaria_gp = venda_diaria_gp.sort_values(by=['Data'])

# tranforming the date for brazilian standard
venda_diaria_gp['Data'] = venda_diaria_gp['Data'].dt.strftime('%d/%m/%Y')

# formating the df so the columns can be the dates
venda_diaria_gp = venda_diaria_gp.pivot_table(index='Data', columns='Clinica', values='Valor Vendido no Dia (somente número):', aggfunc='sum').reset_index()

# Calculate total - select all columns except 'Data'
vendas_cols = [col for col in venda_diaria_gp.columns if col != 'Data']
venda_diaria_gp['Total'] = venda_diaria_gp[vendas_cols].sum(axis=1)

# getting the mentee goals 
metas_por_mentorado = df_final.groupby(['Clinica']).agg({'Qual a sua Meta de Faturamento?' : 'first','Valor Vendido no Dia (somente número):' : 'sum'}).reset_index()

metas_por_mentorado["Meta_dia_util"] = (metas_por_mentorado['Qual a sua Meta de Faturamento?'] - metas_por_mentorado['Valor Vendido no Dia (somente número):']) / num_business_day

venda_diaria_gp, metas_por_mentorado


Clinica,Data,ALMEIDA E GIRON,ATUAL ESTÉTICA E IMAGEM PESSOAL,BARRUFINI E VIDIGAL LTDA - ESPAÇO LASER,BEAUTY GLOW ACADEMY,BLUE CLINIC LTDA,CENTRO DE ESTÉTICA DANIELA DA ROSA,CLAUDIA PELUCO ESTÉTICA E SAÚDE LTDA,CLÍNICA DRA JESSICA CRUZ,CLÍNICA KAYA,...,INSTITUTO AZURI DE ESTETICA,INSTITUTO DERME,INSTITUTO PELE E SAUDE,JESSILY RIBAS TORRES DE AZEVEDO LTDA,LA FEMME LTDA,MITRAS ESTÉTICA,MSP ESTETICA LTDA,RAFAELA FERNANDES SERVIOS MÉDICOS,T DANTAS DOS SANTOS LTDA,TTDS ODONTOLOGIA E EDUCACAO LTDA
0,01/07/2025,597,669,1400,637,968,624,505,1439,1174,...,661,662,819,1085,679,1794,1142,507,2200,1129
1,02/07/2025,616,1183,1044,1058,648,1191,802,1126,2266,...,863,866,1052,920,765,1690,1072,1179,1413,574
2,03/07/2025,760,575,2438,519,1178,737,567,1330,2429,...,1146,959,723,967,581,1698,1045,571,1358,1172
3,04/07/2025,1127,707,2121,1198,710,706,684,2098,2348,...,822,642,510,1060,1078,1782,1041,686,2459,1153
4,05/07/2025,1057,728,1187,863,1082,617,959,1070,1257,...,1115,1109,1094,919,604,1833,1070,543,1643,1146
5,06/07/2025,1059,897,2253,656,869,750,788,1625,1304,...,1036,1124,941,1082,698,2015,566,893,2241,749
6,07/07/2025,831,850,2371,1168,846,771,1021,1978,1388,...,1080,819,1175,1020,813,2052,652,1172,1774,568
7,08/07/2025,590,1055,1688,1121,599,1120,666,1319,1547,...,543,537,1004,545,656,1122,539,775,1434,572
8,09/07/2025,800,638,1310,1129,833,892,1062,2404,1297,...,519,701,596,551,763,2351,1139,769,1181,639
9,10/07/2025,1157,919,2045,520,662,1097,1134,1664,1647,...,1197,584,589,1176,558,2403,586,734,2265,545


In [3]:
#Principais KPI´s 
df_data_kpi = df_final.copy()

df_data_kpi['Vendas Dias ticket'] = df_data_kpi['Valor Vendido no Dia (somente número):']

df_data_kpi = df_data_kpi.groupby('Clinica').agg({'Qual a sua Meta de Faturamento?':'first',
                                                  'Quantidade de Pedidos Gerados no DIa:' : 'sum',
                                          'Valor Vendido no Dia (somente número):':'sum',
                                          'Vendas Dias ticket' : 'mean',
                                          'Atendimentos Realizados no dia.\n(considerando Avaliação)':'sum',
                                          'Avaliações Realizadas no Dia:' : 'sum',
                                          'Leads Gerados no Dia:' : 'sum'}).reset_index()


df_data_kpi = df_data_kpi.rename(columns={'Qual a sua Meta de Faturamento?':'Meta',
                                          'Quantidade de Pedidos Gerados no DIa:' : 'Pedidos',
                                'Valor Vendido no Dia (somente número):' : 'Faturamento Total',
                                'Vendas Dias ticket' : 'Ticket Médio',
                                'Atendimentos Realizados no dia.\n(considerando Avaliação)' : "Atendimento No mês",
                                'Avaliações Realizadas no Dia:' : 'Avaliações no Mês',
                                'Leads Gerados no Dia:' : 'Leads no Mês'})


df_top_kpi = df_data_kpi[["Clinica",'Meta','Pedidos','Faturamento Total','Ticket Médio']].copy()

df_top_kpi['Gap/Plus'] = df_top_kpi['Faturamento Total'] - df_top_kpi['Meta']

df_top_kpi['Atingimento de Meta (%)'] = df_top_kpi['Faturamento Total'] / df_top_kpi['Meta'] * 100

df_top_kpi['Atingimento de Meta (%)'] = df_top_kpi['Atingimento de Meta (%)'].round(2)
df_top_kpi['Ticket Médio'] = df_top_kpi['Ticket Médio'].round(2)

df_top_kpi

,Clinica,Meta,Pedidos,Faturamento Total,Ticket Médio,Gap/Plus,Atingimento de Meta (%)
0,ALMEIDA E GIRON,50000,219,23019,767.30,-26981,46.04
1,ATUAL ESTÉTICA E IMAGEM PESSOAL,50000,229,25361,845.37,-24639,50.72
2,BARRUFINI E VIDIGAL LTDA - ESPAÇO LASER,50000,221,52249,1741.63,2249,104.50
3,BEAUTY GLOW ACADEMY,50000,239,26133,871.10,-23867,52.27
4,BLUE CLINIC LTDA,50000,207,26422,880.73,-23578,52.84
5,CENTRO DE ESTÉTICA DANIELA DA ROSA,50000,241,24949,831.63,-25051,49.90
6,CLAUDIA PELUCO ESTÉTICA E SAÚDE LTDA,50000,225,26178,872.60,-23822,52.36
7,CLÍNICA DRA JESSICA CRUZ,50000,227,51355,1711.83,1355,102.71
8,CLÍNICA KAYA,50000,231,55205,1840.17,5205,110.41
9,DENT.AI,50000,210,24654,821.80,-25346,49.31


In [4]:
# Funil: 
df_data_funil = df_final.copy()

df_data_funil['Vendas Dias ticket'] = df_data_funil['Valor Vendido no Dia (somente número):']

df_data_funil = df_data_funil.rename(columns={
    'Qual a sua Meta de Faturamento?': 'Meta',
    'Quantidade de Pedidos Gerados no DIa:': 'Pedidos',
    'Valor Vendido no Dia (somente número):': 'Faturamento Total',
    'Vendas Dias ticket': 'Ticket Médio',
    'Atendimentos Realizados no dia.\n(considerando Avaliação)': "Atendimento No mês",
    'Avaliações Realizadas no Dia:': 'Avaliações no Mês',
    'Leads Gerados no Dia:': 'Leads no Mês'
})

list_of_clincs = df_data_funil['Clinica'].unique().tolist()

list_of_funil_df = []

for c in list_of_clincs:
    df_clinica = df_data_funil[df_data_funil['Clinica'] == c]
    
    # Agrupando por clínica e agregando as principais métricas
    df_agrupado = df_clinica.agg({
        'Leads no Mês': 'sum',
        'Atendimento No mês': 'sum',
        'Avaliações no Mês': 'sum',
        'Pedidos': 'sum',
        'Ticket Médio': 'mean',
        'Faturamento Total': 'sum'
    }).reset_index()
    
    df_agrupado.columns = ['Métrica', 'Valor']
    
    # Adicionando coluna de Atingimento
    df_agrupado['Atingimento'] = '-'
    
    # Calculando as conversões
    for i, row in df_agrupado.iterrows():
        if row['Métrica'] == 'Atendimento No mês':
            leads = df_agrupado[df_agrupado['Métrica'] == 'Leads no Mês']['Valor'].values[0]
            if leads != 0:
                df_agrupado.at[i, 'Atingimento'] = row['Valor']/leads
                
        elif row['Métrica'] == 'Avaliações no Mês':
            atendimentos = df_agrupado[df_agrupado['Métrica'] == 'Atendimento No mês']['Valor'].values[0]
            if atendimentos != 0:
                df_agrupado.at[i, 'Atingimento'] = row['Valor']/atendimentos
                
        elif row['Métrica'] == 'Pedidos':
            avaliacoes = df_agrupado[df_agrupado['Métrica'] == 'Avaliações no Mês']['Valor'].values[0]
            if avaliacoes != 0:
                df_agrupado.at[i, 'Atingimento'] = row['Valor']/avaliacoes
    
    # Adicionando a clínica como primeira linha
    clinica_row = pd.DataFrame([['Clinica', c, '-']], columns=['Métrica', 'Valor', 'Atingimento'])
    df_agrupado = pd.concat([clinica_row, df_agrupado], ignore_index=True)
    
    list_of_funil_df.append(df_agrupado)

# Para acessar o DataFrame formatado da terceira clínica (índice 2)
list_of_funil_df[2]

,Métrica,Valor,Atingimento
0,Clinica,CLÍNICA KAYA,-
1,Leads no Mês,855.0,-
2,Atendimento No mês,238.0,0.278363
3,Avaliações no Mês,310.0,1.302521
4,Pedidos,231.0,0.745161
5,Ticket Médio,1840.166667,-
6,Faturamento Total,55205.0,-
